<a href="https://colab.research.google.com/github/Rishabh672003/Learning-machine-learning/blob/main/jupyter-notebooks/fetal_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the data and pre-processing it

In [ ]:
!pip install -q wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 45.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
!curl -LO https://physionet.org/static/published-projects/ctu-uhb-ctgdb/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38.2M  100 38.2M    0     0   646k      0  0:01:00  0:01:00 --:--:--  643k


In [ ]:
!unzip /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip
!rm /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip

Archive:  /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1296.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1277.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1499.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1432.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1006.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/2023.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1294.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1336.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1302.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1219.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1218.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/2031.hea  
 extracting: 

In [ ]:
!ls -la /content/__MACOSX/database/signals

ls: cannot access '/content/__MACOSX/database/signals': No such file or directory


In [ ]:
!git clone https://github.com/fabiom91/CTU-CHB_Physionet.org.git

Cloning into 'CTU-CHB_Physionet.org'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 31 (delta 11), reused 16 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 31.69 MiB | 32.74 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [ ]:
!mkdir /content/CTU-CHB_Physionet.org/physionet_data
!mkdir -p signals
!cp -r /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0/* /content/CTU-CHB_Physionet.org/physionet_data

In [ ]:
import pandas as pd
import wfdb
from os import listdir
from tqdm import tqdm

def get_all_records():
    rec_list = []
    for file in listdir("/content/CTU-CHB_Physionet.org/physionet_data"):
        rec = file[:file.find('.')]
        try:
            rec = int(rec)
            rec_list.append(rec)
        except:
            pass
    rec_list = [str(i) for i in rec_list]
    return rec_list

def create_signals_database(rec):
    sample = wfdb.rdsamp("/content/CTU-CHB_Physionet.org/physionet_data/%s" % rec)
    df = pd.DataFrame(sample[0], columns=['FHR','UC'])
    df.index.name = 'seconds'
    df.to_csv('signals/%s.csv' % rec)

def create_ann_dataframe(rec):
    sample = wfdb.rdsamp("/content/CTU-CHB_Physionet.org/physionet_data/%s" % rec)
    ann = sample[1]['comments'][1:]
    ann_dic = {}
    for a in ann:
        if '--' in a:
            ann.remove(a)

    for a in ann:
        key = a[:a.find('  ')]
        if a.find('  ') == -1:
            key = a[:a.find(' ')]
        inv = a[::-1]
        value = inv[:inv.find(' ')][::-1]
        value = float(value)
        ann_dic[key] = [value]

    df1 = pd.DataFrame.from_dict(ann_dic).T
    df1 = df1.rename(columns={0:rec})
    return df1

def append_ann_dataframes(df,df1):
    rec = df1.columns[0]
    df[rec] = df1[rec]
    return df


df = pd.DataFrame()
for rec in tqdm(get_all_records()):
    create_signals_database(rec)
    df = append_ann_dataframes(df,create_ann_dataframe(rec))
df.to_csv('ann_db.csv')

print('DONE!')

 10%|▉         | 106/1104 [00:06<00:32, 30.63it/s]<ipython-input-8-a2da56fbc565>:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
<ipython-input-8-a2da56fbc565>:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
<ipython-input-8-a2da56fbc565>:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

DONE!


# Starting the work on dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/ann_db.csv")

In [ ]:
df.head()

,Unnamed: 0,1365,2035,1204,1073,1217,1176,1441,1108,1474,...,1093,1349,1306,1232,1326,2017,1224,1211,1355,1167
0,pH,7.39,7.26,7.32,7.13,7.18,7.22,7.23,7.02,7.27,...,7.27,7.32,7.39,7.23,7.29,7.32,7.33,6.98,7.27,7.36
1,BDecf,3.32,2.23,3.68,7.11,4.57,7.80,5.27,12.81,3.43,...,0.01,0.89,4.14,3.00,1.84,-0.16,2.72,NaN,4.06,5.03
2,pCO2,4.60,7.20,5.60,8.40,8.20,6.10,6.80,8.50,6.60,...,7.70,6.40,4.40,7.60,6.70,6.70,5.70,NaN,6.40,4.60
3,BE,-4.50,-4.30,-4.90,-9.80,-6.70,-9.60,-6.90,-15.50,-5.30,...,-2.20,-1.80,-4.10,-5.40,-3.80,-2.00,-4.10,NaN,-5.60,-5.70
4,Apgar1,9.00,8.00,9.00,10.00,7.00,8.00,9.00,7.00,10.00,...,9.00,9.00,10.00,8.00,9.00,10.00,9.00,6.00,9.00,9.00


In [ ]:
df.shape

(35, 553)

In [ ]:
df.head(20)

,Unnamed: 0,1365,2035,1204,1073,1217,1176,1441,1108,1474,...,1093,1349,1306,1232,1326,2017,1224,1211,1355,1167
0,pH,7.39,7.26,7.32,7.13,7.18,7.22,7.23,7.02,7.27,...,7.27,7.32,7.39,7.23,7.29,7.32,7.33,6.98,7.27,7.36
1,BDecf,3.32,2.23,3.68,7.11,4.57,7.80,5.27,12.81,3.43,...,0.01,0.89,4.14,3.00,1.84,-0.16,2.72,NaN,4.06,5.03
2,pCO2,4.60,7.20,5.60,8.40,8.20,6.10,6.80,8.50,6.60,...,7.70,6.40,4.40,7.60,6.70,6.70,5.70,NaN,6.40,4.60
3,BE,-4.50,-4.30,-4.90,-9.80,-6.70,-9.60,-6.90,-15.50,-5.30,...,-2.20,-1.80,-4.10,-5.40,-3.80,-2.00,-4.10,NaN,-5.60,-5.70
4,Apgar1,9.00,8.00,9.00,10.00,7.00,8.00,9.00,7.00,10.00,...,9.00,9.00,10.00,8.00,9.00,10.00,9.00,6.00,9.00,9.00
5,Apgar5,10.00,9.00,9.00,10.00,8.00,9.00,10.00,8.00,10.00,...,10.00,10.00,10.00,8.00,9.00,10.00,9.00,8.00,10.00,10.00
6,NICU days,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Seizures,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,HIE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Intubation,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
new_cols = pd.DataFrame(columns=['index','FHR_mean','UC_mean'])
for i in range(1001, 2046):
    f = f"signals/{i}.csv"
    try:
        with open(f):
          df = pd.read_csv(f)

          fhr, uc = df["FHR"].mean(), df["UC"].mean()
          new_cols.loc[-1] = [i, fhr, uc]
          new_cols.index += 1
          new_cols = new_cols.sort_index()
    except:
        continue

In [ ]:
new_cols.head()

,index,FHR_mean,UC_mean
0,2045.0,96.125571,14.546898
1,2044.0,115.084445,15.543302
2,2043.0,82.792328,18.059265
3,2042.0,90.791597,39.765486
4,2041.0,105.066601,16.282025
